In [2]:
import cv2
import math
import numpy as np
import pandas as pd
from pathlib import Path


IMAGE_PATHS = [
    "nlt1.jpg","nlt2.jpg",
    "nrt1.jpg","nrt2.jpg","nrt3.jpg","nrt4.jpg","nrt5.jpg","nrt6.jpg","nrt7.jpg"
]
SCALE_PCT = 30
BOX_W, BOX_H = 275, 200
OUT_XLSX = "angles_output.xlsx"
OUT_PNG = "Via_Angles.png"


def imread_resized(path, scale_pct):
    img = cv2.imread(str(path))
    if img is None:
        print(f"[WARN] Not found or unreadable: {path}")
        return None
    w = int(img.shape[1] * scale_pct / 100)
    h = int(img.shape[0] * scale_pct / 100)
    return cv2.resize(img, (w, h), interpolation=cv2.INTER_AREA)

def hstack_with_sidebar(images, box_w, box_h):
    max_h = max(im.shape[0] for im in images)
    total_w = sum(im.shape[1] for im in images) + box_w
    canvas = np.full((max_h + box_h, total_w, 3), 255, np.uint8)
    x = 0
    for im in images:
        canvas[:im.shape[0], x:x+im.shape[1]] = im
        x += im.shape[1]
    return canvas

def angle_deg(p1, p2, p3):
    v1 = (p2[0]-p1[0], p2[1]-p1[1])
    v2 = (p3[0]-p1[0], p3[1]-p1[1])
    a = v1[0]*v2[0] + v1[1]*v2[1]
    m1 = math.hypot(*v1); m2 = math.hypot(*v2)
    if m1 == 0 or m2 == 0:
        return 0.0
    c = max(-1.0, min(1.0, a/(m1*m2)))
    return math.degrees(math.acos(c))

def write_excel(inside, outside, avg_in, avg_out, path):
    try:
        df = pd.DataFrame({
            "No": np.arange(1, len(inside)+1),
            "Inside Angles (°)": inside,
            "Outside Angles (°)": outside,
            "Average Inside Angle (°)": [avg_in]*len(inside),
            "Average Outside Angle (°)": [avg_out]*len(outside),
        })
        df.to_excel(path, index=False)

        try:
            from openpyxl import load_workbook
            wb = load_workbook(path)
            sh = wb.active
            if len(inside) >= 1:
                sh.merge_cells(start_row=2, start_column=4, end_row=len(inside)+1, end_column=4)
                sh.merge_cells(start_row=2, start_column=5, end_row=len(outside)+1, end_column=5)
            wb.save(path)
        except Exception as e:
            print(f"[INFO] openpyxl merge skipped: {e}")
    except Exception as e:
        print(f"[ERR] Excel write failed: {e}")


class AngleApp:
    def __init__(self, image_paths):
        imgs = []
        for p in image_paths:
            im = imread_resized(Path(p), SCALE_PCT)
            if im is not None:
                imgs.append(im)
        if not imgs:
            raise SystemExit("[FATAL] No images loaded. Check names/paths.")

        self.base = hstack_with_sidebar(imgs, BOX_W, BOX_H)
        self.canvas = self.base.copy()
        self.zoom = 1.0
        self.zoom_step = 0.1
        self.points = []
        self.inside = []
        self.outside = []
        self.mark_id = 1
        self.sidebar_x0 = self.base.shape[1] - BOX_W
        self.refresh_sidebar()

    def refresh_sidebar(self):
        self.canvas[:, self.sidebar_x0:] = 255
        y = 30
        for i, ang in enumerate(self.inside, start=1):
            cv2.putText(self.canvas, f"Angle {i}: {ang:.2f} deg",
                        (self.sidebar_x0 + 10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0,0,0), 2)
            y += 30
        if self.inside:
            avg_in = sum(self.inside)/len(self.inside)
            cv2.putText(self.canvas, f"AoA: {avg_in:.2f} deg",
                        (self.sidebar_x0 + 10, y + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (0,0,255), 2)

    def add_point(self, pt):
        n = len(self.points)
        if n % 3 == 0:
            cv2.circle(self.canvas, pt, 10, (0,0,255), cv2.FILLED)
            cv2.putText(self.canvas, str(self.mark_id), (pt[0]-5, pt[1]+5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)
            self.mark_id += 1
        else:
            p0 = tuple(self.points[(n//3)*3])
            cv2.line(self.canvas, p0, pt, (255,0,0), 2)
        self.points.append(list(pt))

        if len(self.points) % 3 == 0:
            p1, p2, p3 = self.points[-3:]
            ang = angle_deg(p1, p2, p3)
            in_ang = min(ang, 360-ang)
            out_ang = 360 - in_ang
            self.inside.append(in_ang)
            self.outside.append(out_ang)
            self.refresh_sidebar()
            avg_in = sum(self.inside)/len(self.inside)
            avg_out = sum(self.outside)/len(self.outside)
            write_excel(self.inside, self.outside, avg_in, avg_out, OUT_XLSX)

    def on_mouse(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            zx = int(x / self.zoom)
            zy = int(y / self.zoom)
            zx = np.clip(zx, 0, self.canvas.shape[1]-1)
            zy = np.clip(zy, 0, self.canvas.shape[0]-1)
            self.add_point((int(zx), int(zy)))

    def run(self):
        win = "Combined Images"
        cv2.namedWindow(win, cv2.WINDOW_NORMAL)
        cv2.setMouseCallback(win, self.on_mouse)

        while True:
            view_w = int(self.canvas.shape[1]*self.zoom)
            view_h = int(self.canvas.shape[0]*self.zoom)
            view = cv2.resize(self.canvas, (view_w, view_h), interpolation=cv2.INTER_LINEAR)
            cv2.imshow(win, view)
            try:
                cv2.imwrite(OUT_PNG, view)
            except Exception as e:
                print(f"[WARN] PNG save failed: {e}")

            k = cv2.waitKey(1) & 0xFF
            if k in (ord('+'), ord('=')):
                self.zoom += self.zoom_step
            elif k == ord('-'):
                self.zoom = max(0.1, self.zoom - self.zoom_step)
            elif k == ord('r'):
                self.canvas = self.base.copy()
                self.points.clear(); self.inside.clear(); self.outside.clear()
                self.mark_id = 1
                self.refresh_sidebar()
            elif k == ord('q'):
                break

        cv2.destroyAllWindows()

if __name__ == "__main__":
    try:
        AngleApp(IMAGE_PATHS).run()
    except Exception as e:
        print(f"[FATAL] {e}")
